# Deep Learning - Practicum 4 - Generative Adversarial Networks

**Topics covered**: Generative Adversarial Networks(GAN) with Multi-layer Perceptron

**Deliverables**:
- Complete the tasks as detailed in this document.  <br>
This practice uses [Keras](https://keras.io/api/) with Tensorflow.

**Objectives**:
Usually a GAN is referring to DCGAN (deep convolution GAN): a GAN where the generator and discriminator are deep convnets. Training a DCGAN is computational cost. In this practice, deep convnets will be replaced with multi-layer perceptron. <br>
In this practice, we're to implement a standard GAN using a multi-layer perceptron for both the discriminator and generator. We train the GAN on MNIST images. After completing the tasks, you'll be get familiaried with the schematic GAN implementation. <br>

---
Import needed packages

In [ ]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras import layers, optimizers
from keras.datasets import mnist
import matplotlib.pyplot as plt

## 1. Introduction
Schematically, the GAN we're to implement looks like this: <br>
- 1 A generator network maps vectors of shape (latent_dim,) to images of shape (28, 28) which is the size of images from MNIST. <br>
- 2 A discriminator network maps images of shape (28, 28) to a binary score estimating the probability that the image is real. <br>
- 3 A gan network chains the generator and the discriminator together: gan(x) = discriminator(generator(x)). Thus this gan network maps latent space vectors to the discriminator’s assessment of the realism of these latent vectors as decoded by the generator. <br>
- 4 We train the discriminator using examples of real and fake images along with “real”/“fake” labels, just as you train any regular image-classification model. <br>
- 5 To train the generator, you use the gradients of the generator’s weights with regard to the loss of the gan model. This means, at every step, you move the weights of the generator in a direction that makes the discriminator more likely to classify as “real” the images decoded by the generator. In other words, you train the generator to fool the discriminator.

The settings of our GAN mode is given in the cell below.

In [ ]:
latent_dim = 100
img_size = 28 * 28   # MNIST images

num_epochs = 100
batch_size = 128

## 2. Prepare Data

In [ ]:
# load mnist data, only need train_data, the task is not for recognition, it doesn't need target
(x_train, _), (_, _) = mnist.load_data()

# Data Normalization
# Your code goes here


# We're using multi-layered model, reshape image matrix into a vector (784,)
# Your code goes here


# The shape of the normalized and reshaped data set is (60000, 784)
print(x_train.shape)

## 3. Generator
First, let’s develop a generator model that turns a vector (from the latent space—during training it will be sampled at random) into a candidate image. <br>
- input shape: latent_dim <br>
- output shape: img_size <br>
Build up a 3-layer feed forward neural network. <br>
Suggested structure (you don't have to follow it) is as following: <br>
- hidden layer: 128 neuron, `LeakyReLU()`, `Dropout(0.5)`
- output layer: `tanh()`

In [ ]:
# generator
# Your code goes here


## 4. Discriminator
Next, we’ll develop a discriminator model that takes as input a candidate image (real or synthetic) and classifies it into one of two classes: “generated image” or “real image that comes from the training set.” <br>
Build up a 3-layer feed forward neural network and compile it. <br>
Suggested structure (you don't have to follow it) is as follow: <br>
- hidden layer: 128 neuron, `LeakyReLU()`, `Dropout(0.5)` <br>
- output layer: `sigmod()` (It is compulsory that use sigmod for classification). <br>
- optimizer: `RMSprop(lr=0.0001, decay=1e-8)` <br>
- loss: `'binary_crossentropy'`

In [ ]:
# discriminator
# Your code goes here


## 5. GAN
Finally, we’ll set up the GAN, which chains the generator and the discriminator. <br>
When trained, this model will move the generator in a direction that improves its ability to fool the discriminator. This model turns latent-space points into a classification decision—“fake” or “real”—and it’s meant to be trained with labels that are always “these are real images.” So, training gan will update the weights of generator in a way that makes discriminator more likely to predict “real” when looking at fake images. <br>
It’s very important to **note** that you set the discriminator to be frozen during training (non-trainable): its weights won’t be updated when training gan. <br>
Run the code in cell below to set up the GAN.

In [ ]:
# GAN

discriminator.trainable = False                    # set the discriminator to be frozen
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)
gan_optimizer = keras.optimizers.RMSprop(lr=0.0001, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

## 6. Train the Model
Now we can begin training. For each epoch, do the following: <br>
- 1 Draw random points in the latent space (random noise). <br>
- 2 Generate images with generator using this random noise. <br>
- 3 Mix the generated images with real ones. <br>
- 4 Train discriminator using these mixed images, with corresponding targets: either “real” (for the real images) or “fake” (for the generated images). <br>
- 5 Draw new random points in the latent space. <br>
- 6 Train gan using these random vectors, with targets that all say “these are real images.” This updates the weights of the generator (only, because the discriminator is frozen inside gan) to move them toward getting the discriminator to predict “these are real images” for generated images: this trains the generator to fool the discriminator. <br>

In [ ]:
losses = {"D":[], "G":[]}

batchCount = int(x_train.shape[0] / batch_size)

# After successfully completeing the code in this cell, and hitting the run button
# go and grab a cup of tea or coffee, it'll take a while if you're using CPU

for epoch in range(num_epochs):
    for _ in range(batchCount):
        
        ###################################################################
        ####  Preparing real images and fake/generated images         #####
        ###################################################################
    
        # Create a batch by drawing random index numbers from the training set x_train         
        real_images = # Put your code here
    
        # Create noise vectors for the generator
        # TRICK, use normal distribution not a uniform distribution
        random_latent_vectors = # Put your code here
        
        # Use generator.predict() to generate the images from the noise
        generated_images = # put your code here
        
        # Combine the two types of images
        combined_images = np.concatenate([generated_images, real_images])
    
        # Labels: genteratred - 0; real - 1
        labels = # put your code here
        # IMPORTANT TRICK: Adds random noise to the labels
        labels += 0.05 * np.random.random(labels.shape)
        
        ###################################################################
        ####  Train discriminator and generator                       #####
        ###################################################################
    
        # Train the discriminator on the combined_images and labels
        d_loss = # put your code here
        
        
        # Train the generator        
        # randomly generate the input, use normal distribution
        random_latent_vectors = # put your code here
        
        # Create labels that say “these are all real images”  - it’s a lie! In order to fool the discriminator
        misleading_targets = # put your code here
        
        # Train the generator via the GAN model, where the weights of generator are frozen
        g_loss = # put your code here
        
        # store losses for each batch
        losses["D"].append(d_loss)
        losses["G"].append(g_loss)
        
    # plot 10 images from generated samples every 20 epoch
    # Your code goes here


## 7. Plot the learning curve
Plot the loss of discriminator and generator respectively. Generator loss should be decreasing along with the iteration number, while discriminator loss is increasing.

In [ ]:
# Plot loss
# Your code goes here
